# TFM Suministro Global - CREAR RELACIONES DE ETAPAS SUMINISTRO: 
##    > CREAR relación  PAIS--[EXTRAE] -->  MaterialB
## _Neo4j - py2neo_
**OBSERVACIONES**  
Ya creados los nodos de PAIS y de MaterialB


## Partimos de la existencia, en paralelo a esta ejecución, de una instancia levantada y accesible de Neo4j  
### La primera vez que se quiera usar _py2neo_ erá  necesario asegurar su instalación:  _!pip install py2neo_

In [1]:
##!pip install py2neo 

In [1]:
#importar la librería para poder hacer uso de sus funcionalidades y objetos
from py2neo import *

In [2]:
#acceder a la instancia levantada de Neo4j - en nuestro caso instancia Docker - asignamos credenciales de autentificación -
graph_test01 = Graph(auth=('neo4j', 's3cr3t'))

In [3]:
#que es el objeto - graph_test01
graph_test01

Graph('bolt://localhost:7687')

In [4]:
#que es el objeto - tx
# tx = graph_test01.begin()

In [5]:
# tx

In [6]:
#query de conexión de prueba sin errores
# verificar existencia de NODOS previos creados
#--- cuenta los nodos con diferentes tipos de etiqueta y los une en una única salida, 1 fila por cada tipo (name, cnt)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
"
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,116
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [8]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:REFINA]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal"       
                                  
graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442
2,REFINA,475


## CARGAR DATOS

In [33]:
import numpy as np    #datos y operativa álgegra y matrices de n dimensiones (arrays)
import pandas as pd   #carga dataset y manipulación de datos  y dataframes

In [100]:
#DATOS: Fichero completo de ETAPAS DE SUMINISTRO DE MATERIALES BRUTOS (Etapa extracción) Y REFINADOS (Etapa refinado/procesado)
#       dentro del análisis de suministro crítico GLOBAL actual - Atendiendo al alcance del estudio EU CRM 2020
#FICHERO: GLEtapas12_cargar.csv
#CONTENIDO: Para cada Material, Etapa de producción 1-Extracción/Minería y  2-Refinado/procesado, 
#           Pais qye produce ese material en cada etapa que aplique, cuota de producción 
#           de ese pais y material en esa etapa e índice de trade entre 0,80 y 1,78 que da idea de la facilidad de trading  
#           para ese material en esa etapa y para ese país productor
#
#USO: Crear las RELACIONES entre "Pais" --[REFINA]--> "MaterialREF"
#     ** Propiedades de [REFINA]: 
#           SHR_PaisMat: proporción decimal de 2 dígitos (entre 0 y 1) de la cuota de producción 
#                        de ese pais para ese material en esa etapa de producción (identificada por la relación)  
#           t_PaisMat: indice de trading de 2 decimales entre 0,80 y 1,78 de esa producción,
#                      para ese pais ,  ese material y  esa etapa de producción (identificada por la relación)
#
# Ruta del fichero al Github del proyecto - TFM
rutaFichero =  "https://raw.githubusercontent.com/JuanKFD/UNIR_TFM/main/GLEtapas12_cargar.csv" 
dfEtapas12 = pd.read_csv(rutaFichero, sep=';')


In [35]:
dfEtapas12.head()

,Material,Pais1,Share1,WGI1,t1,MatRefinado,Pais2,Share2,WGI2,t2
0,Aluminium,na,NaN,NaN,NaN,Aluminium,China,"0,52","5,83","1,10"
1,Aluminium,na,NaN,NaN,NaN,Aluminium,Russian Federation,"0,06","6,20","1,10"
2,Aluminium,na,NaN,NaN,NaN,Aluminium,Other non EU,"0,08","0,00","1,00"
3,Aluminium,na,NaN,NaN,NaN,Aluminium,Canada,"0,05","2,26","1,00"
4,Aluminium,na,NaN,NaN,NaN,Aluminium,United Arab Emirates,"0,04","3,94","1,00"


In [36]:
dfEtapas12.tail()

,Material,Pais1,Share1,WGI1,t1,MatRefinado,Pais2,Share2,WGI2,t2
1126,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1127,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1128,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1129,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN
1130,Zirconium,na,NaN,NaN,NaN,Zirconium,na,NaN,NaN,NaN


###  IMPORTANTE .-   
En este dataset, tanto en la etapa 1 (etiquetas terminadas en 1) de producción como en la etapa 2 (etiquetas terminadas en 2)  
existen valores de Pais1 o Pais2 con el valor "na" (no aplica) . Esto indica que no debe considerarse esa línea de etapa
Así, cuando un Pais sea "na" se pasará de inmediato a procesar la siguiente línea de esa etapa.  
**El procesado de este dataset** se realizará por separado para cada etapa:
etapa 1-Extracción/Minería genera las **relaciones de tipo ("Extrae")** y para cada Pais genera un Material Bruto (MaterialB) y
etapa 2-Refinado/Procesado genera las **relaciones de tipo ("Refina")** y para cada Pais genera un Material Refinado (MaterialREF)

In [37]:
dfEtapas12.shape

(1131, 10)

In [38]:
dfEtapas12.columns

Index(['Material', 'Pais1', 'Share1', 'WGI1', 't1', 'MatRefinado', 'Pais2',
       'Share2', 'WGI2', 't2'],
      dtype='object')

**Cargados en memoria las 1131 líneas** de relación **para cada etapa**, con  _pais productor, material producido, cuota de ese país en esa producción e índice de trading t para esa etapa productiva-pais-material, atendiendo a los datos de base de partida del análisis de **cadena de suministro global** (EU CRM 2020)

# PREVISUALIZACION DE DATOS - USOS

### Con el fin de procesar de manera más parametrizada y como presentación de otra manera de tratarlo, creo bucle en base a los propios datos de las columnas

In [101]:
## en caso de que un valor sea float, se transforma a cadena para la rápida visualización concatenada como string
strSeparator = "|"
for i in range(0, dfEtapas12.shape[0]):
    strVar = ""
    for j in range(0, dfEtapas12.shape[1]):
        if type(dfEtapas12[dfEtapas12.columns[j]][i]) == float: 
            strAux= str(dfEtapas12[dfEtapas12.columns[j]][i])
        else:
            strAux= dfEtapas12[dfEtapas12.columns[j]][i]
        strVar += strAux
        strVar += strSeparator
    print(strVar) 
# se visualiza con el separador específico identificado en la variable  - strSeparator

Aluminium|na|nan|nan|nan|Aluminium|China|0,52|5,83|1,10|
Aluminium|na|nan|nan|nan|Aluminium|Russian Federation|0,06|6,20|1,10|
Aluminium|na|nan|nan|nan|Aluminium|Other non EU|0,08|0,00|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Canada|0,05|2,26|1,00|
Aluminium|na|nan|nan|nan|Aluminium|United Arab Emirates|0,04|3,94|1,00|
Aluminium|na|nan|nan|nan|Aluminium|India|0,04|5,45|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Australia|0,03|2,36|1,00|
Aluminium|na|nan|nan|nan|Aluminium|United States|0,03|2,92|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Norway|0,02|2,03|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Brazil|0,02|5,08|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Bahrain|0,02|5,15|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Iceland|0,01|2,52|1,00|
Aluminium|na|nan|nan|nan|Aluminium|South Africa|0,01|4,65|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Saudi Arabia|0,01|5,51|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Qatar|0,01|4,03|1,00|
Aluminium|na|nan|nan|nan|Aluminium|Mozambique|0,01|5,93|1,00|
Aluminium|na|na

In [87]:
if np.isnan(dfEtapas12.Share1[0]):
    print("null")

null


# CREAR RELACION ENTRE  PAIS y Material Bruto (MaterialB)
### RELACIÓN: _Pais--[EXTRAE]--> MaterialB_
**Observaciones:**   
(1) Nodos previamente creados    
(2) Si para una etapa E el valor de PaisE = "na", (siendo E 1 o 2), el proceso de generación avanzará a la fila siguiente 
    puesto que la línea "na" en Pais no contiene valores. Se respeta este formato para conciliar más facilmente con los datos       de origen (excel capturada a partir de los mismos datos en el PDF oficial final y abierto del trabajo Europeo CRM 2020).
(3) Las dos relaciones (las dos etapas) contienen _DOS propiedades_:  

**SHR_PaisMat**: proporción decimal de 2 dígitos (entre 0 y 1) de la cuota de producción de ese pais 
                para ese material en esa etapa de producción (identificada por la relación)  
**t_PaisMat**: indice de trading de 2 decimales entre 0,80 y 1,78 de esa producción,
                para ese pais ,  ese material y  esa etapa de producción (identificada por la relación)
relación _EXTRAE_: Etapa1 de Extracción/Minería
relación _REFINA_: Etapa2 de Refinado/Procesado del material

### Consulta patrón _CQL-Neo4j_ para crear la relación buscada
**MATCH**   
    (a:**Pais**),  
    (b:**MaterialB**)  

**WHERE** a.name = _"Paisxxx"_ **AND** b.name = _"Materialxxx"_  
**CREATE** (a)**-[r:EXTRAE {SHR_PaisMat: 0.nn, t_PaisMat: m.mm }]->**(b)  
**RETURN** type(r)

In [102]:
nFilas = dfEtapas12.shape[0]
nFilas 

1131

In [103]:
#recordar las columnas del dataset 
dfEtapas12.columns

Index(['Material', 'Pais1', 'Share1', 'WGI1', 't1', 'MatRefinado', 'Pais2',
       'Share2', 'WGI2', 't2'],
      dtype='object')

In [104]:
#  Crear RELACION --EXTRAE-->   Una fila por cada relación, ETAPA 1, contiene el Pais productor y el material producido, 
# MaterialB, así como las propiedades de cuota de producción de ese país en esta etapa y el índice de trading asociado  
# (previamente creados ambos nodos) 
#  Nodos: Pais -- para ser asignado como productor de un material para la etapa a tratar
#  Nodos: MaterialB -- para ser asignado como producido por un Pais
## PREVISAULIZAR QUERYs antes de la ejecución (visualizar 2 de muestra)
TipoNodoA = "(a:Pais)"
TipoNodoB = "(b:MaterialB)"
TipoRelacion = " :EXTRAE" 
inicio = 0
nFilas = 50    # aluminio no tiene extracción porque se obtiene de la Bauxita; llegamos a 50 para llegar al "Antimony"
PaisNA = "na" #valor en Pais que indica saltar línea y procesar la siguiente de esa etapa
#
# --- Procesar primeras  filas nFilas vinculadas al primer material, el "Aluminio". 
#    Si es ok, seguir con el resto (1131 - 50 ) en las siguientes celdas para esta etapa.
#    Cuando el pais es "na" -no aplica esa fila- salta la línea y procesa la siguiente
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais1[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais1[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.Material[i]                       \
            + "'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + str(dfEtapas12.Share1[i]).replace(',', '.')          \
            + ", t_PaisMat: "                                      \
            + str(dfEtapas12.t1[i]).replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        print("QRY [" + str(i) + "]> "+ strQry)
    else:
        continue

# querys  visualizadas


QRY [27]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.74, t_PaisMat: 1.10} ]->(b) RETURN type(r)
QRY [28]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Tajikistan' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [29]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [30]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Myanmar' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
QRY [31]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Bolivia' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.10} ]->(b) RETURN type(r)
QRY [32]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.02,

In [105]:
#comprobar que sigue abierta la conexión para enviar las queries a la BD 
graph_test01

Graph('bolt://localhost:7687')


# Ejecutar  creación de relaciones iniciales. Si es OK en Neo4j, lanzar el resto 

In [106]:
# Crear la RELACIÓN en la BD , una por CADA FILA ---- primeras filas ----
#relación :EXTRAE  (COLUMNAS xxxxxx1 - ETAPA 1)
## query para crea la relación con sus propiedades 
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais1[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais1[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.Material[i]                        \
            + "'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + str(dfEtapas12.Share1[i]).replace(',', '.')              \
            + ", t_PaisMat: "                                      \
            + str(dfEtapas12.t1[i]).replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        #
        # ejecutar Query en BD abierta - Neo4j
        #
        graph_test01.run(strQry)
        #Visualizar Query ejecutada 
        print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
    else:
        continue
# elementos creados en BD

EJECUTADA QRY [27]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.74, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [28]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Tajikistan' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [29]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [30]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Myanmar' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [31]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Bolivia' AND b.name = 'Antimony' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [32]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.

In [107]:
# guardar como dataframe el resultado de buscar todas las relaciones creadas de tipo USADO_EN
# de momento sólo están creadas las 2 primeras, y etiquetamos el resultado con el nombre de la relación 
dfGraphMat= graph_test01.run("MATCH p=()-[r:EXTRAE]->() RETURN (p) as EXTRAE").data()

In [108]:
# visualizar los datos creados usando .data() perteneciente a la funcionalidad de la librería py2neo
dfGraphMat


[{'EXTRAE': Path(Node('Pais', Pais_iso3='CHN', name='China'), EXTRAE(Node('Pais', Pais_iso3='CHN', name='China'), Node('MaterialB', Mat_Abrev='Sb', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Antimony'), SHR_PaisMat=0.74, t_PaisMat=1.1))},
 {'EXTRAE': Path(Node('Pais', Pais_iso3='TJK', name='Tajikistan'), EXTRAE(Node('Pais', Pais_iso3='TJK', name='Tajikistan'), Node('MaterialB', Mat_Abrev='Sb', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Antimony'), SHR_PaisMat=0.08, t_PaisMat=1.0))},
 {'EXTRAE': Path(Node('Pais', Pais_iso3='RUS', name='Russian Federation'), EXTRAE(Node('Pais', Pais_iso3='RUS', name='Russian Federation'), Node('MaterialB', Mat_Abrev='Sb', Mat_Familia='General', Mat_Subfamilia='Other non-ferrous metals', name='Antimony'), SHR_PaisMat=0.04, t_PaisMat=1.0))},
 {'EXTRAE': Path(Node('Pais', Pais_iso3='MMR', name='Myanmar'), EXTRAE(Node('Pais', Pais_iso3='MMR', name='Myanmar'), Node('MaterialB', Mat_Abrev='Sb', Mat_Fami

In [109]:
#Recordar las dimensiones del dataset 
dfEtapas12.shape

(1131, 10)

### Creamos el resto de las relaciones para las restantes filas  
una vez que se verifica la creación de las n primeras filas

In [110]:
dfEtapas12.shape[0]

1131

In [111]:
nFilas

50

In [112]:
# Crear la RELACIÓN en la BD , una por CADA FILA (****restantes ***) 
#relación EXTRAE
## query para crea la relación con sus propiedades (en este caso no aplica)
#
inicio = nFilas
nFilas = dfEtapas12.shape[0]
# Crear la RELACIÓN en la BD , una por CADA FILA ---- primeras filas ----
#relación :EXTRAE (COLUMNAS xxxxxx1 - ETAPA 1)
## query para crea la relación con sus propiedades 
#
for i in range(inicio, nFilas):
    if dfEtapas12.Pais1[i] != PaisNA:
        strQry = "MATCH "                                    \
            + TipoNodoA + ", " + TipoNodoB                   \
            + " WHERE a.name = '"                             \
            +  dfEtapas12.Pais1[i]                            \
            + "'"                                             \
            + " AND b.name = '"                                 \
            +  dfEtapas12.Material[i]                        \
            + "'"                                            \
            + " CREATE (a)-[r" + TipoRelacion                   \
            + " { SHR_PaisMat: "                                  \
            + str(dfEtapas12.Share1[i]).replace(',', '.')              \
            + ", t_PaisMat: "                                      \
            + str(dfEtapas12.t1[i]).replace(',', '.')                 \
            + "} ]->(b)"                                          \
            + " RETURN type(r)"                                 
        #
        # ejecutar Query en BD abierta - Neo4j
        #
        graph_test01.run(strQry)
        #Visualizar Query ejecutada 
        print("EJECUTADA QRY [" + str(i) + "]> "+ strQry)
    else:
        continue
# elementos creados en BD# elementos  creados

EJECUTADA QRY [59]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Baryte' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.38, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [60]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'India' AND b.name = 'Baryte' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.12, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [61]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Morocco' AND b.name = 'Baryte' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.10, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [62]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Iran, Islamic Rep.' AND b.name = 'Baryte' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [63]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Kazakhstan' AND b.name = 'Baryte' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.07, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [64]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Turkey' AND b.name = 'Baryte'

EJECUTADA QRY [115]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Other non EU' AND b.name = 'Beryllium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [126]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Turkey' AND b.name = 'Borate' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.43, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [127]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b.name = 'Borate' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.25, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [128]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Chile' AND b.name = 'Borate' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.11, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [129]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Argentina' AND b.name = 'Borate' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.07, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [130]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Peru' AND b.name = '

EJECUTADA QRY [210]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.name = 'Coking coal' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.16, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [211]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Coking coal' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.07, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [212]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b.name = 'Coking coal' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.06, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [213]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'India' AND b.name = 'Coking coal' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [214]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Canada' AND b.name = 'Coking coal' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [215]> MATCH (a:Pais), (b:MaterialB) WHERE a.

EJECUTADA QRY [294]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Thailand' AND b.name = 'Erbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [295]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Malaysia' AND b.name = 'Erbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [297]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Europium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.86, t_PaisMat: 1.59} ]->(b) RETURN type(r)
EJECUTADA QRY [298]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.name = 'Europium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.06, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [299]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b.name = 'Europium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [300]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation'

EJECUTADA QRY [367]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Malaysia' AND b.name = 'Gadolinium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [382]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Gold' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.14, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [383]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Other non EU' AND b.name = 'Gold' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.09, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [384]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.name = 'Gold' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [385]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Gold' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.08, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [386]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b

...

EJECUTADA QRY [681]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Myanmar' AND b.name = 'Natural Teak wood' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.09, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [682]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Ghana' AND b.name = 'Natural Teak wood' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [683]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Nigeria' AND b.name = 'Natural Teak wood' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [684]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Other non EU' AND b.name = 'Natural Teak wood' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [685]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Thailand' AND b.name = 'Natural Teak wood' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [686]> MATCH (a:Pais), (b:M

EJECUTADA QRY [755]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Morocco' AND b.name = 'Phosphate rock' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.11, t_PaisMat: 1.10} ]->(b) RETURN type(r)
EJECUTADA QRY [756]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b.name = 'Phosphate rock' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.10, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [757]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Phosphate rock' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.06, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [758]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Peru' AND b.name = 'Phosphate rock' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.06, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [759]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Brazil' AND b.name = 'Phosphate rock' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [760]> MATCH (a:Pais), (b:Materia

EJECUTADA QRY [859]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Netherlands' AND b.name = 'Silica sand' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.20, t_PaisMat: 0.80} ]->(b) RETURN type(r)
EJECUTADA QRY [860]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Turkey' AND b.name = 'Silica sand' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.05, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [861]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Italy' AND b.name = 'Silica sand' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.05, t_PaisMat: 0.80} ]->(b) RETURN type(r)
EJECUTADA QRY [862]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Malaysia' AND b.name = 'Silica sand' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.04, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [863]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'France' AND b.name = 'Silica sand' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 0.80} ]->(b) RETURN type(r)
EJECUTADA QRY [864]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'India' 

EJECUTADA QRY [1003]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Terbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.86, t_PaisMat: 1.59} ]->(b) RETURN type(r)
EJECUTADA QRY [1004]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Australia' AND b.name = 'Terbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.06, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1005]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United States' AND b.name = 'Terbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1006]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Terbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.02, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1007]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'India' AND b.name = 'Terbium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1008]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Brazil'

EJECUTADA QRY [1056]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'United Kingdom' AND b.name = 'Tungsten' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.01, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1068]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'China' AND b.name = 'Vanadium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.39, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1069]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'South Africa' AND b.name = 'Vanadium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.32, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1070]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Russian Federation' AND b.name = 'Vanadium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.25, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1071]> MATCH (a:Pais), (b:MaterialB) WHERE a.name = 'Brazil' AND b.name = 'Vanadium' CREATE (a)-[r :EXTRAE { SHR_PaisMat: 0.03, t_PaisMat: 1.00} ]->(b) RETURN type(r)
EJECUTADA QRY [1072]> MATCH (a:Pais), (b:MaterialB) WHERE a.name 

In [113]:
#validamos que todos los nodos se han creado (hasta este paso)
queryPRB= "MATCH (n)                                    \
RETURN labels (n)[0] as name, count (n) as Ntotal       \
UNION MATCH (n)                                         \
WITH labels (n)[1] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN name, Ntotal                                     \
UNION MATCH (n)                                         \
WITH labels (n)[2] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[3] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal                                    \
UNION MATCH (n)                                         \
WITH labels (n)[4] as name, count (n) as Ntotal         \
WHERE name IS NOT NULL                                  \
RETURN  name, Ntotal"  
                                  
graph_test01.run(queryPRB).to_data_frame()

,name,Ntotal
0,MaterialB,78
1,MaterialREF,78
2,Pais,116
3,GrupoAplicacion,88
4,SectorAplicacion,25


In [114]:
#validamos que todas las RELACIONES se han creado (hasta este paso)
queryPRB= "MATCH n=()-[r:INCLUIDO_EN]->()              \
RETURN type(r) as TipoRelacion, count (r) as Ntotal    \
UNION MATCH m=()-[p:USADO_EN]->()                      \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:REFINA]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal    \
UNION MATCH m=()-[p:EXTRAE]->()                        \
RETURN type(p) as TipoRelacion, count (p) as Ntotal" 

graph_test01.run(queryPRB).to_data_frame()

,TipoRelacion,Ntotal
0,INCLUIDO_EN,442
1,USADO_EN,442
2,REFINA,475
3,EXTRAE,699


# fin proceso - relación  "EXTRAE" - CREADA

In [81]:
# fin archivo